# DS 3000 Take-Home Coding Exam 

## Diego Cicotoste

In [2]:
name = "Diego Cicotoste"
print(f'I, {name}, declare that the following work is entirely my own, and that I did not copy or seek help from any students who have currently or previously taken this course, nor from any online source (incl. AI) other than private messages between myself and the professor on Piazza/via email.')

I, Diego Cicotoste, declare that the following work is entirely my own, and that I did not copy or seek help from any students who have currently or previously taken this course, nor from any online source (incl. AI) other than private messages between myself and the professor on Piazza/via email.


Due: Sunday Oct 13 @ 11:59 PM EST

**Please block out about 100 minutes of time to complete this exam**

### Submission Instructions
Submit this `ipynb` file to Gradescope (this can also be done via the assignment on Canvas).  To ensure that your submitted `ipynb` file represents your latest code, make sure to give a fresh `Kernel > Restart & Run All` just before uploading the `ipynb` file to gradescope. **In addition:**
- "Sign" the academic integrity pledge above with your name
- Make sure you comment your code effectively
- If problems are difficult for the TAs/Profs to grade, you will lose points

### Note: This is an **EXAM**
While I encourage some dialogue (*never* sharing of answers) on most assignments, this is an **EXAM**. You should treat is as such; there is **NO COLLABORATION** allowed. If anyone is found to have copied any of their answers on this exam from any source, they will receive a 0, be reported to OSCCR, and potentially fail the course. Also:
- You are welcome to post a private note on piazza, but to keep a consistent testing environment for all students we are unlikely to provide assistance.
- You may not contact other students with information about this this quiz
    - even saying "it was easy/hard" in a general sense can introduce a bias in favor of students who take the quiz earlier or later
- Under no circumstances should you share a copy of this quiz with anyone who isn't a member of the course staff.

### Tips for success
- Start early
- Remember to use cells and headings to make the notebook easy to read (if a grader cannot find the answer to a problem, you will receive no points for it)
- Under no circumstances may one student view or share their ungraded homework or quiz with another student [(see also)](http://www.northeastern.edu/osccr/academic-integrity)


In [57]:
# the following modules may be helpful in completing the exam
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

# Web Scraping and Cleaning Korean Dramas

Your goal is to build a data frame that includes three columns: `category`, `movie/show`, and `year` based on the 70 best Korean Dramas according to [this website](https://www.marieclaire.com/culture/a26895105/best-korean-dramas/).

## Part 1 (25 points)

Use the `requests` module and the `BeautifulSoup` function to scrape the website and then study the html to determine what tags you should use to refine your soup. **Hint:** there should be a single tag which you should be able to use `soup.find_all()` to grab a list that contains all the information we need.

In [34]:
url = "https://www.marieclaire.com/culture/a26895105/best-korean-dramas/"

data = {
    "movie_show": [],
    "year": []
}

response = requests.get(url)
html = response.text

soup = BeautifulSoup(html, 'html.parser')
for h2 in soup.find_all('h2'):
    title = h2.text.strip()
    title = title.replace("'", "")
    if '(' and ')' in title:
        show, year = title.rsplit('(', 1)
        year = year.replace(')', '')
    else:
        show = title
        year = 'N/A'
    
    data["movie_show"].append(show.strip())
    data["year"].append(year)


In [35]:
df = pd.DataFrame(data)
df

,movie_show,year
0,Action/Thriller Korean Dramas,N/A
1,All of Us Are Dead,2022–
2,D.P,2021–2023
3,Stay In The Know,N/A
4,Happiness,2021
...,...,...
73,Reply 1988,2015–2016
74,Something in the Rain,2018
75,Twenty Five Twenty One,2022
76,When the Camellia Blooms,2019


## Part 2 (25 points)

Create two empty lists, `categories` and `movies_shows`. Use the list of tags you generated from Part 1 to fill those lists out with strings such that they are the same length and that each movie in `movies_shows` is matched with its correct category in `categories`.

In [55]:
categories = []

category = ""
for cat in soup.find_all("h2"):
    if "article-body__section" in cat.get("class", []):
        category = cat.text
    categories.append(category)

data["Category"] = categories
df = pd.DataFrame(data)
df


,movie_show,year,Category
0,Action/Thriller Korean Dramas,N/A,Action/Thriller Korean Dramas
1,All of Us Are Dead,2022–,Action/Thriller Korean Dramas
2,D.P,2021–2023,Action/Thriller Korean Dramas
3,Stay In The Know,N/A,Action/Thriller Korean Dramas
4,Happiness,2021,Action/Thriller Korean Dramas
...,...,...,...
73,Reply 1988,2015–2016,Slice of Life Korean Dramas
74,Something in the Rain,2018,Slice of Life Korean Dramas
75,Twenty Five Twenty One,2022,Slice of Life Korean Dramas
76,When the Camellia Blooms,2019,Slice of Life Korean Dramas


## Part 3 (25 points)

Notice that the strings contained in the `categories` and `movies` lists you created in Part 2 are still somewhat messy, and that we have yet to create the `year` list. Make sure you clean the data so that:

- The categories do not have the `' Korean Dramas'` part of their string.
- The years are extracted from the movies/shows and stored in their own list as integers.
    - **Note 1:** If you extracted the movies and shows the way I did, the third movie/show `Stay In The Know` is not actually a movie or a show, but rather the newsletter signup header. It does not have a date, and should be removed and its corresponding category also removed. Check out the [list.pop function docs](https://docs.python.org/3/tutorial/datastructures.html), but be careful with it!
    - **Note 2:** some of the actual shows are still going on; only grab the **first** year for each movie/show.

In [96]:
df['Category'] = df['Category'].str.replace('Korean Dramas', '').str.strip()

cleaned_years = []
cleaned_movies_shows = []
cleaned_categories = []

for index in range(len(df)):
    category = df['Category'][index]
    movie_show = df["movie_show"][index]

    year_str = df.at[index, 'year'].strip()
    year = year_str[:4] if year_str[:4].isdigit() else 'N/A'

    if year != 'N/A' and category != movie_show:
        cleaned_years.append(year)
        cleaned_movies_shows.append(movie_show)
        cleaned_categories.append(category)

df_clean = pd.DataFrame({
    "Category": cleaned_categories,
    "movie_show": cleaned_movies_shows,
    "year": cleaned_years
})

df_clean


,Category,movie_show,year
0,Action/Thriller,All of Us Are Dead,2022
1,Action/Thriller,D.P,2021
2,Action/Thriller,Happiness,2021
3,Action/Thriller,Healer,2014
4,Action/Thriller,Moving,2023
...,...,...,...
65,Slice of Life,Reply 1988,2015
66,Slice of Life,Something in the Rain,2018
67,Slice of Life,Twenty Five Twenty One,2022
68,Slice of Life,When the Camellia Blooms,2019


## Part 4 (25 points)

Put your three cleaned lists (`categories`, `movies_shows`, and `year`) into a data frame and output the `.head()`. Then, calculate the average year for each category. Discuss, in a markdown cell:
1. if it seems that certain categories are represented by more recent movies/shows than others (explicitly reference the numerical summary in your answer)
2. what other features might be useful to collect about the movies and shows in order to better understand what makes them popular

In [97]:
df_clean.head()

,Category,movie_show,year
0,Action/Thriller,All of Us Are Dead,2022
1,Action/Thriller,D.P,2021
2,Action/Thriller,Happiness,2021
3,Action/Thriller,Healer,2014
4,Action/Thriller,Moving,2023


Romance            1.555539e+50
Slice of Life      1.681001e+46
Action/Thriller    1.838366e+42
Fantasy            1.838365e+42
Melodrama          2.527752e+30
Historical         2.524003e+30
Professional       2.890289e+26
dtype: float64